In [22]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [23]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [24]:
jan_df['duration'] = jan_df.tpep_dropoff_datetime - jan_df.tpep_pickup_datetime
jan_df.duration = jan_df.duration.apply(lambda td: td.total_seconds() / 60)

print(np.std(jan_df.duration))
before = jan_df.shape
print(before)
jan_df = jan_df[(jan_df.duration >= 1) & (jan_df.duration <= 60)]
after = jan_df.shape
print(after)
print((before[0] - after[0])/before[0] * 100)

42.5943442974141
(3066766, 20)
(3009173, 20)
1.8779717787402104


Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [25]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

jan_df[categorical] = jan_df[categorical].astype(str)



dv = DictVectorizer()

train_dicts = jan_df[categorical + numerical].to_dict(orient='records')
print(train_dicts)
X_train = dv.fit_transform(train_dicts)
print(X_train.ndim)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



2


In [26]:
print(train_dicts)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data

In [27]:
lr = LinearRegression()
y_train = jan_df.duration
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)
print(rmse)

7.658396474418433


In [28]:
print(X_train.shape)

(3009173, 516)


Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [29]:
feb_df['duration'] = feb_df.tpep_dropoff_datetime - feb_df.tpep_pickup_datetime
feb_df.duration = feb_df.duration.apply(lambda td: td.total_seconds() / 60)

feb_df = feb_df[(feb_df.duration >= 1) & (feb_df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

feb_df[categorical] = feb_df[categorical].astype(str)


val_dicts = feb_df[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_pred = lr.predict(X_val)
y_val = feb_df.duration
rmse = root_mean_squared_error(y_val, y_pred)
print(rmse)

7.820267421957621
